# Exploración de las SVMs

**NOTA: Para este ejercicio estamos más interesados en explorar el algoritmo.  Por lo tanto nos saltearemos las partes de normalización y de división de los datos en entrenamiento/prueba.  Esto nos permitirá poner más atención en cómo pueden cambiar los parámetros cambiar un SVM**

[Enlace a un excelente artículo sobre SVM](http://citeseerx.ist.psu.edu/viewdoc/download;jsessionid=4448154647BC7B10C991CEF2236BBA38?doi=10.1.1.114.4288&rep=rep1&type=pdf)
* A tutorial on support vector regression by ALEX J. SMOLA and BERNHARD SCHOLKOPF

## SVM - Clasificación

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Datos

Los datos que se utilizarán simulan una investigación en medicina en la que ratones infectados con un virus fueron tratados con varias dosis de dos medicinas y luego examinados dos semanas más tarde para ver si aún seguían infectados. Con estos datos, el objetivo es crear un modelo de clasificación que pueda predecir (dadas dos medidas de dosis) si el ratón seguirá infectado con el virus. 

Podrán darse cuenta que los grupos son muy "separables", esto es a propósito, para explorar el comportamiento de los varios parámetros de un modelo SVM.

In [ ]:
datos = pd.read_csv("mouse_viral_study.csv")

In [ ]:
datos.head()

In [ ]:
datos.columns

## Clases

In [ ]:
sns.scatterplot(x = 'Med_1_mL', y = 'Med_2_mL', hue = 'Virus Present',
                data = datos, palette = 'seismic')

## El Hiperplano de separación

El objetivo con SVM es la creación del hiperplano que mejor logre separar las categorías.  En 2 dimensiones esto es simplemente una línea.

In [ ]:
sns.scatterplot(x = 'Med_1_mL', y = 'Med_2_mL', hue = 'Virus Present', 
                palette = 'seismic', data = datos)

# De alguna forma se quiere que mágicamente se cree una línea de separación ( una línea en 2D)

x = np.linspace(0, 10, 100)
m = -1
b = 11
y = m * x + b
plt.plot(x, y, 'k')

## SVM - Support Vector Machine

In [ ]:
from sklearn.svm import SVC # Supprt Vector Classifier

In [ ]:
help(SVC)

**NOTA: Parar este ejemplo, se explorará el algoritmo, por lo tanto no se hará normalización ni división de datos entrenamiento/prueba**

In [ ]:
y = datos['Virus Present']
X = datos.drop('Virus Present', axis = 1) 

In [ ]:
modelo = SVC(kernel = 'linear', C = 1000)
modelo.fit(X, y)

In [ ]:
# Esta importación viene de un módulo externo .py
# https://scikit-learn.org/stable/auto_examples/svm/plot_separating_hyperplane.html
from svm_margin_plot import plot_svm_boundary

In [ ]:
plot_svm_boundary(modelo, X, y)

## Hiper Parámetros

### C

Parámetro de regularization. La potencia de la regurlarización es **inversamente** proporcional a C. Debe ser estrictamente positivo. La penalización es de l2 cuadrado.

*Nota: Si está más interesado en el desarrollo matemático, el valor de C como se describre en  ISLR, C en scikit-learn ss **inversamente** proporcional a este valor.*

In [ ]:
modelo = SVC(kernel = 'linear', C = 0.05)
modelo.fit(X, y)

In [ ]:
plot_svm_boundary(modelo, X, y)

### Kernel

[Selección de un "Kernel"](https://stats.stackexchange.com/questions/18030/how-to-select-kernel-for-svm?rq=1)

#### rbf - [Radial Basis Function](https://en.wikipedia.org/wiki/Radial_basis_function_kernel)


Cuando se entrena una SVM con Radial Basis Function (RBF) kernel, es necesario considerar dos parámetros: C y gamma. El parámetro C, común a todos los kernels SVM, da un balance (trades off) la mala clasificación de los ejemplos de entrenamientocontra la simplicidad de la superficie de decisión. Un valor bajo de C hace que la superficie de decisión sea lisasurface, un valor alto de C busca classificar todas las observaciones de entrenamiento correctamente. gamma define qué tanta influence tiene una solo observación de entrenamieto. Entre más alto sea gamma, más cerca deben estar las otras muestras para que sean afectadas.

In [ ]:
modelo = SVC(kernel = 'rbf', C = 1)
modelo.fit(X, y)
plot_svm_boundary(modelo, X, y)

In [ ]:
modelo = SVC(kernel = 'sigmoid')
modelo.fit(X, y)
plot_svm_boundary(modelo, X, y)

#### Grado (Degree) (sólo para kernels poly)

El grado (Degree) de la función polinomial del kernel ('poly').
Este parámetro es ignorado por todos los demás modelos.

In [ ]:
modelo = SVC(kernel = 'poly', C = 1, degree = 1)
modelo.fit(X, y)
plot_svm_boundary(modelo, X, y)

In [ ]:
modelo = SVC(kernel = 'poly', C = 1, degree = 2)
modelo.fit(X, y)
plot_svm_boundary(modelo, X, y)

### gamma

gamma : {'scale', 'auto'} o float, default='scale'
    Coeficient de Kernel para 'rbf', 'poly' y 'sigmoid'.

    - si ``gamma = 'scale'`` (default) es pasado cuando utiliza
      1 / (n_features * X.var()) como valor de gamma,
    - si 'auto', utiliza 1 / n_features.

In [ ]:
modelo = SVC(kernel = 'rbf', C = 1, gamma = 0.01)
modelo.fit(X, y)
plot_svm_boundary(modelo, X, y)

## Búsqueda por malla (Grid Search)

Hay que tener en mente que, para este ejemplo simple, las categorías fueron fácilmente separadas. Esto quiere decir que cada variación de modelo podría fácilmente llegar a una exactitud de 100%.  Por lo tanto el significado de una búsqueda por malla no tiene sentido.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
svm = SVC()
malla_parametros = {'C':[0.01, 0.1, 1],'kernel':['linear', 'rbf']}
malla = GridSearchCV(svm, malla_parametros)

In [ ]:
# Nota:  de nuevo no se usó Train|Test
malla.fit(X, y)

In [ ]:
# Exactitud de 100% (como se esperaba)
malla.best_score_

In [ ]:
malla.best_params_

De nuevo, este ejercicio fue más para ver el proceso de búsqueda por malla.  Recuerde que en una situación real, sí deben realizar una división entreno/prueba y obtener métricas finales de evaluación.